In [1]:
# import findspark
# findspark.init("/home/exouser/nyc_collisions/spark-3.5.3-bin-hadoop3/")

import pyspark
# from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, to_date, concat_ws, hour, count, desc, sum, month, year, dayofweek, expr

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1731795914375_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# spark = SparkSession.builder.appName("crash analysis").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data = spark.read.csv("Motor_Vehicle_Collisions_-_Crashes_20241014.csv", header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
data.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']

# Data Cleaning

### - Drop Duplicates

In [5]:
unique_df = data.distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
data.exceptAll(unique_df).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+-------+--------+--------+---------+--------+--------------+-----------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|BOROUGH|ZIP CODE|LATITUDE|LONGITUDE|LOCATION|ON STREET NAME|CROSS STREET NAME|OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VEHICLE 1|CONTRIBUTING FACTOR VEHICLE 2|CONTRIBUTING F

In [7]:
data.groupby("COLLISION_ID").count().filter("count > 1").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|COLLISION_ID|count|
+------------+-----+
|        NULL|    2|
+------------+-----+

In [8]:
data.filter(data["COLLISION_ID"].isNull()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+-------+--------+----------+-----------+--------------------+--------------+-----------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|     CRASH DATE|CRASH TIME|BOROUGH|ZIP CODE|  LATITUDE|  LONGITUDE|            LOCATION|ON STREET NAME|CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VE

In [9]:
data = data.dropna(subset=["COLLISION_ID"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
data.filter(data["CRASH DATE"].isNull()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+-------+--------+--------+---------+--------+--------------+-----------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|BOROUGH|ZIP CODE|LATITUDE|LONGITUDE|LOCATION|ON STREET NAME|CROSS STREET NAME|OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VEHICLE 1|CONTRIBUTING FACTOR VEHICLE 2|CONTRIBUTING F

In [11]:
data.filter(data["CRASH TIME"].isNull()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+-------+--------+--------+---------+--------+--------------+-----------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|BOROUGH|ZIP CODE|LATITUDE|LONGITUDE|LOCATION|ON STREET NAME|CROSS STREET NAME|OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VEHICLE 1|CONTRIBUTING FACTOR VEHICLE 2|CONTRIBUTING F

### - Convert Data Types

In [12]:
data = data.withColumn("NUMBER OF PERSONS INJURED", col("NUMBER OF PERSONS INJURED").cast("int"))
data = data.withColumn("NUMBER OF PERSONS KILLED", col("NUMBER OF PERSONS KILLED").cast("int"))

data = data.withColumn("NUMBER OF PEDESTRIANS INJURED", col("NUMBER OF PEDESTRIANS INJURED").cast("int"))
data = data.withColumn("NUMBER OF PEDESTRIANS KILLED", col("NUMBER OF PEDESTRIANS KILLED").cast("int"))

data = data.withColumn("NUMBER OF MOTORIST INJURED", col("NUMBER OF MOTORIST INJURED").cast("int"))
data = data.withColumn("NUMBER OF MOTORIST KILLED", col("NUMBER OF MOTORIST KILLED").cast("int"))

data = data.withColumn("NUMBER OF CYCLIST INJURED", col("NUMBER OF CYCLIST INJURED").cast("int"))
data = data.withColumn("NUMBER OF CYCLIST KILLED", col("NUMBER OF CYCLIST KILLED").cast("int"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
data = data.withColumn("CRASH DATE", to_date(col("CRASH DATE"), "MM/dd/yyyy"))
data = data.withColumn("CRASH TIMESTAMP", to_timestamp(concat_ws(" ", col("CRASH DATE"), col("CRASH TIME")), "yyyy-MM-dd H:mm"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### - Review Data Quality

In [14]:
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CRASH DATE: date (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: stri

In [15]:
data.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------+--------+---------+----------+-----------------------+-----------------------+-----------------+-----------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|BOROUGH |ZIP CODE|LATITUDE |LONGITUDE |LOCATION               |ON STREET NAME         |CROSS STREET NAME|OFF STREET NAME        |NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF 

# Temporal Analysis

### - What time of day has the highest number of collisions?

In [16]:
hourly_collisions = data \
    .withColumn("HOUR", hour(col("CRASH TIMESTAMP"))) \
    .groupBy("HOUR") \
    .agg(count("*").alias("TOTAL COLLISIONS")) \
    .orderBy(col("TOTAL COLLISIONS").desc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
hourly_collisions.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------+
|HOUR|TOTAL COLLISIONS|
+----+----------------+
|  16|          152561|
|  17|          149320|
|  14|          141107|
|  15|          132617|
|  18|          131349|
|  13|          122138|
|   8|          116587|
|  12|          116397|
|   9|          112203|
|  11|          109461|
|  19|          108110|
|  10|          104834|
|  20|           90420|
|  21|           77003|
|  22|           70224|
|   0|           69118|
|   7|           65058|
|  23|           59047|
|   6|           47642|
|   1|           37291|
+----+----------------+
only showing top 20 rows

In [18]:
# Example: Export results for visualization in Pandas
hourly_collisions.toPandas().plot(kind='bar', x='HOUR', y='TOTAL COLLISIONS')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Pandas >= 1.0.5 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1731795914375_0008/container_1731795914375_0008_01_000001/pyspark.zip/pyspark/sql/pandas/conversion.py", line 86, in toPandas
    require_minimum_pandas_version()
  File "/mnt/yarn/usercache/livy/appcache/application_1731795914375_0008/container_1731795914375_0008_01_000001/pyspark.zip/pyspark/sql/pandas/utils.py", line 34, in require_minimum_pandas_version
    raise ImportError(
ImportError: Pandas >= 1.0.5 must be installed; however, it was not found.



### - Are there seasonal trends in motor vehicle collisions?

In [19]:
monthly_collisions = data \
    .withColumn("MONTH", month(col("CRASH DATE"))) \
    .groupBy("MONTH") \
    .agg(count("*").alias("TOTAL COLLISIONS")) \
    .orderBy("MONTH")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
monthly_collisions.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------+
|MONTH|TOTAL COLLISIONS|
+-----+----------------+
|    1|          164124|
|    2|          151515|
|    3|          169715|
|    4|          157397|
|    5|          180765|
|    6|          181722|
|    7|          192385|
|    8|          189918|
|    9|          190111|
|   10|          190155|
|   11|          178551|
|   12|          179404|
+-----+----------------+

### - How do collision rates change over the years?

In [21]:
yearly_collisions = data \
    .withColumn("YEAR", year(col("CRASH DATE"))) \
    .groupBy("YEAR") \
    .agg(count("*").alias("TOTAL COLLISIONS")) \
    .orderBy("YEAR")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
yearly_collisions.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------+
|YEAR|TOTAL COLLISIONS|
+----+----------------+
|2012|          100545|
|2013|          203742|
|2014|          206039|
|2015|          217693|
|2016|          229831|
|2017|          231007|
|2018|          231564|
|2019|          211486|
|2020|          112916|
|2021|          110555|
|2022|          103886|
|2023|           96599|
|2024|           69899|
+----+----------------+

### - What are the patterns in accidents on weekdays vs. weekends?

In [23]:
weekday_collisions = data \
    .withColumn("DAY_OF_WEEK", dayofweek(col("CRASH DATE"))) \
    .groupBy("DAY_OF_WEEK") \
    .agg(count("*").alias("TOTAL COLLISIONS")) \
    .orderBy("DAY_OF_WEEK")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
weekday_collisions.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+
|DAY_OF_WEEK|TOTAL COLLISIONS|
+-----------+----------------+
|          1|          256138|
|          2|          303639|
|          3|          312870|
|          4|          310437|
|          5|          317381|
|          6|          338510|
|          7|          286787|
+-----------+----------------+

# Driver Behavior and Contributing Factors

In [25]:
# Combine all contributing factor columns into a single column
factors = data.select(
    expr("stack(5, `CONTRIBUTING FACTOR VEHICLE 1`, `CONTRIBUTING FACTOR VEHICLE 2`, `CONTRIBUTING FACTOR VEHICLE 3`, `CONTRIBUTING FACTOR VEHICLE 4`, `CONTRIBUTING FACTOR VEHICLE 5`) as FACTOR")
).filter((col("FACTOR").isNotNull()) & (col("FACTOR") != "Unspecified"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Group by contributing factors and count occurrences
common_factors = factors.groupBy("FACTOR") \
    .agg(count("*").alias("TOTAL OCCURRENCES")) \
    .orderBy(col("TOTAL OCCURRENCES").desc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
common_factors.show(10, truncate=False)  # Display top 10 most common factors

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+-----------------+
|FACTOR                        |TOTAL OCCURRENCES|
+------------------------------+-----------------+
|Driver Inattention/Distraction|526380           |
|Failure to Yield Right-of-Way |144190           |
|Following Too Closely         |136240           |
|Other Vehicular               |102450           |
|Backing Unsafely              |86318            |
|Passing or Lane Usage Improper|73294            |
|Passing Too Closely           |62314            |
|Turning Improperly            |61365            |
|Fatigued/Drowsy               |59349            |
|Unsafe Lane Changing          |48702            |
+------------------------------+-----------------+
only showing top 10 rows

# Save the Data

In [40]:
test = data.select([col(c).alias(c.replace(" ", "_")) for c in data.columns])
test.write \
    .mode("overwrite") \
    .format("parquet") \
    .saveAsTable("motor_vehicle_collisions.collisions")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# Save PySpark DataFrame to S3 as CSV
test.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("s3://nyc-collisions-ecc/collisions/")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…